<a href="https://colab.research.google.com/github/NehaChowdhary/Bit-Rebels/blob/main/Information%20Retrieval%20Models/BM25Plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/aliannejadi/ClariQ.git

Cloning into 'ClariQ'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 220 (delta 19), reused 32 (delta 14), pack-reused 176
Receiving objects: 100% (220/220), 253.36 MiB | 33.61 MiB/s, done.
Resolving deltas: 100% (110/110), done.
Updating files: 100% (41/41), done.


In [ ]:
ls

ClariQ/  data/  README.md  sample_runs/  src/


In [ ]:
!pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from rank_bm25 import *
import nltk
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

def stem_tokenize(text, remove_stopwords=True):
  stemmer = PorterStemmer()
  tokens = [word for sent in nltk.sent_tokenize(text) \
                                      for word in nltk.word_tokenize(sent)]
  tokens = [word for word in tokens if word not in \
          nltk.corpus.stopwords.words('english')]
  return [stemmer.stem(word) for word in tokens]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
cd ClariQ/

/content/ClariQ/sample_runs/ClariQ/ClariQ


In [ ]:
ls

data/  README.md  sample_runs/  src/


In [ ]:
# Files paths
request_file_path = 'data/dev.tsv'
question_bank_path = 'data/question_bank.tsv'
run_file_path = 'sample_runs/dev_bm25'

In [ ]:
# Reads files and build bm25 corpus (index)

dev = pd.read_csv(request_file_path, sep='\t')
question_bank = pd.read_csv(question_bank_path, sep='\t').fillna('')

question_bank['tokenized_question_list'] = question_bank['question'].map(stem_tokenize)
question_bank['tokenized_question_str'] = question_bank['tokenized_question_list'].map(lambda x: ' '.join(x))

bm25_corpus = question_bank['tokenized_question_list'].tolist()
bm25 = BM25Plus(bm25_corpus)

# Runs bm25 for every query and stores output in file.

with open(run_file_path, 'w') as fo:
  for tid in dev['topic_id'].unique():
    query = dev.loc[dev['topic_id']==tid, 'initial_request'].tolist()[0]
    bm25_ranked_list = bm25.get_top_n(stem_tokenize(query, True), 
                                    bm25_corpus, 
                                    n=30)
    bm25_q_list = [' '.join(sent) for sent in bm25_ranked_list]
    preds = question_bank.set_index('tokenized_question_str').loc[bm25_q_list, 'question_id'].tolist()
    for i, qid in enumerate(preds):    
      fo.write('{} 0 {} {} {} bm25\n'.format(tid, qid, i, len(preds)-i))

In [ ]:
cd src

/content/ClariQ/sample_runs/ClariQ/ClariQ/src


In [ ]:
run_file_path = "../sample_runs/dev_bm25"

In [ ]:
# Report question relevance performance
! python clariq_eval_tool.py    --eval_task question_relevance\
                                --data_dir ../data/ \
                                --experiment_type dev \
                                --run_file {run_file_path} \
                                --out_file {run_file_path}_question_relevance.eval

Recall5: 0.3245570421150917
Recall10: 0.5626277940325928
Recall20: 0.6674997108155003
Recall30: 0.6912818698329535


In [ ]:
! python clariq_eval_tool.py    --eval_task document_relevance\
                                --data_dir ../data/ \
                                --experiment_type dev \
                                --run_file {run_file_path} \
                                --out_file {run_file_path}.eval

NDCG1: 0.1859375
NDCG3: 0.1565278915736497
NDCG5: 0.15213811412234102
NDCG10: 0.1330149674169258
NDCG20: 0.12457116254616747
P1: 0.225
P3: 0.18333333333333332
P5: 0.17375000000000002
P10: 0.135625
P20: 0.1128125
MRR100: 0.2979868145159347
